# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1}(\psi) {q} + \nabla \psi = {0}\\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ \psi = 0 \text{ on } \partial_{top} \Omega \qquad \psi = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$
and
$$
k(\psi) = \frac{1}{\rho g}
\begin{bmatrix}
\psi + 1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

In [1]:
import shutil
import os

import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

In [2]:
K = 1000
eps = 1e-6

method = 'newton'

In [3]:
N = 10

rho = 1000
g = pp.GRAVITY_ACCELERATION

In [4]:
output_directory = 'output_stationary'

In [5]:
subdomain = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

In [6]:
key = "flow"

bc_value = []
bc_essential = []

gravity = []

RT0 = pg.RT0(key)
P0  = pg.PwConstants(key)

def g_func(x): return np.array([0, -1, -1])

for subdomain, data in mdg.subdomains(return_data=True):
    g_proj = RT0.interpolate(subdomain, g_func)
    gravity.append(RT0.assemble_mass_matrix(subdomain) @ g_proj)

    # permeability tensor
    perm = pp.SecondOrderTensor(np.ones(subdomain.num_cells))
    parameters = {
        "second_order_tensor": perm,
    }
    pp.initialize_data(subdomain, data, key, parameters)
        
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    left_right = np.logical_or(subdomain.face_centers[0, :] == 0,  subdomain.face_centers[0, :] == 1)
    
    bottom = subdomain.face_centers[1, :] == 0
    top    = subdomain.face_centers[1, :] == 1
    
    def psi_bc(x): return 1

    bc_value.append(- RT0.assemble_nat_bc(subdomain, psi_bc, bottom))

    essential_pressure_dofs = np.zeros(P0.ndof(subdomain), dtype=bool)
    bc_essential.append(np.hstack((left_right, essential_pressure_dofs)))

In [7]:
div = pg.cell_mass(mdg) @ pg.div(mdg)

# get the degrees of freedom for each variable
dof_psi, dof_q = div.shape

# assemble the right-hand side
rhs = np.zeros(dof_psi + dof_q)

rhs[:dof_q] += np.hstack(bc_value)
rhs[:dof_q] += np.hstack(gravity)

In [8]:
def conductivity_tensor(psi):
    return pp.SecondOrderTensor(2 * np.ones(subdomain.num_cells) / (rho * g), np.ones(subdomain.num_cells) / (rho * g), np.ones(subdomain.num_cells) / (rho * g)) # (psi + 1) / (rho * g)

In [9]:
x = np.zeros(dof_psi + dof_q)

for k in range(K):
    print('Iteration ', k)
    pp.initialize_data(subdomain, data, key, {
        "second_order_tensor": conductivity_tensor(x[-dof_psi:]),
    })

    # construct the local matrices
    mass = pg.face_mass(mdg)

    # assemble the saddle point problem
    spp = sps.bmat([[mass, -div.T], 
                    [ div,  None]], format="csc")
    
    # solve the problem
    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc(np.hstack(bc_essential), np.zeros(dof_q + dof_psi))
    
    prev = x
    x = ls.solve()

    if np.sum(np.power(x-prev, 2)) < eps:
        break

Iteration  0
Iteration  1


In [10]:
def export(current_sol) -> None:
    save = pp.Exporter(mdg, "sol", folder_name="output_gravity")
    proj_q = RT0.eval_at_cell_centers(subdomain)
    proj_p =  P0.eval_at_cell_centers(subdomain)

    q = current_sol[:dof_q]
    p = current_sol[-dof_psi:]

    for _, data in mdg.subdomains(return_data=True):
        data[pp.STATE] = {"cell_q": (proj_q @ q).reshape((3, -1), order="F"), 
                          "cell_p": (proj_p @ p)}
    
    save.write_vtu(["cell_q", "cell_p"])

    return

In [11]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [12]:
# post process variables
export(x)